# Hybrid search and multi-query with RAG

In this module, you'll learn about the hybrid search and multi query concepts, and how to apply them using Amazon Bedrock and Knowledge bases for Amazon Bedrock APIs.
This module contains:
1. [Overview](#1-Overview)
2. [Pre-requisites](#2-Pre-requisites)
3. [How hybrid search improves the FM generations?](#3-how-hybrid-search-improves-the-fm-generations)
4. [How to apply multi-queries approach?](#4-how-to-apply-multi-queries-approach)

## Overview

For RAG-based applications, the accuracy of the generated response from Foundation Models (FMs) is dependent on the context provided to the model. Context is retrieved from the vector database based on the user query. Semantic search is widely used because it is able to understand more human-like questions—a user’s query is not always directly related to the exact keywords in the content that answers it. 

Semantic search helps provide answers based on the meaning of the text. However, it has limitations in capturing all the relevant keywords. Its performance relies on the quality of the word embeddings used to represent meaning of the text. To overcome such limitations, you can combine semantic search with keyword search (hybrid search), which improve the relevance of search results.
    

### Hybrid search

Hybrid search takes advantage of the strengths of multiple search algorithms, integrating their unique capabilities to enhance the relevance of returned search results. For RAG-based applications, semantic search capabilities are commonly combined with traditional keyword-based search to improve the relevance of search results. It enables searching over both the content of documents and their underlying meaning.

<!-- ![hybrid search](images/hybrid-overview.png) -->

<img src="images/hybrid-overview.png" width="50%" height="30%">



It works great for RAG-based applications where the retriever has to handle a wide variety of natural language queries. The keywords help cover specific entities in the query such as product name, color, and price, while semantics better understands the meaning and intent within the query. For example, if you have want to build a chatbot for an ecommerce website to handle customer queries such as the return policy or details of the product, using hybrid search will be most suitable.

<b>Benefits: </b>
Both keyword and semantic search will return a separate set of results along with their relevancy scores, which are then combined to return the most relevant results.

## Pre-requisites
Before being able to answer the questions, the documents must be processed and stored in a knowledge base. For this notebook, we use a `Synthetic data` (10K financial reports for fictitious company) to create the Knowledge Base. 

1. Upload your documents (data source) to Amazon S3 bucket.
2. Create the Knowledge Base using [1a_create_ingest_documents_test_kb.ipynb](https://github.com/aws-samples/amazon-bedrock-samples/blob/main/knowledge-bases/01-rag-concepts/1a_create_ingest_documents_test_kb.ipynb)
3. Note the Knowledge Base ID


#### Notes:
1. For our notebook we will use the ```Retrieve``` and ```RetrieveAndGenerate``` APIs provided by Knowledge Bases for Amazon Bedrock. These APIs convert queries into embeddings, searches the knowledge base, and then augment (in case of ```RetrieveAndGenerate``` API) the foundation model prompt with the search results as context information and returns the FM-generated response to the question. 

    -   The output of the ```RetrieveAndGenerate``` API includes the `generated response`, `source attribution` as well as the `retrieved text chunks`.
    -   The output of the ```Retrieve API``` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. 


2. We will use the Anthropic Claude 3 Sonnet model on Amazon Bedrock for response generation.

## Setup

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain--force-reinstall --quiet
%pip install langchain-community --force-reinstall --quiet
%pip install langchain-aws --force-reinstall --quiet

In [8]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Initialize boto3 client and variables
Through out the notebook, we are going to utilise RetrieveAndGenerate and Retrieve APIs to test knowledge base features.

In [9]:
import json
import boto3
import pprint as pp
from botocore.exceptions import ClientError
from botocore.client import Config

kb_id = "Y4NQIKORK8" # replace it with your Knowledge base id.

# Create boto3 session
sts_client = boto3.client('sts')
boto3_session = boto3.session.Session()
region_name = boto3_session.region_name

# Create bedrock agent client
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0}, region_name=region_name)
bedrock_agent_client = boto3_session.client("bedrock-agent-runtime",
                              config=bedrock_config)
# Create bedrock client
bedrock_client = boto3_session.client("bedrock-runtime",
                              config=bedrock_config)

# Define FM to be used for generations 
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # we will be using Anthropic Claude 3 Haiku throughout the notebook
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'


def retrieve_and_generate(query, kb_id, model_arn, max_results, search_type):
    response = bedrock_agent_client.retrieve_and_generate(
            input={
                'text': query
            },
        retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn, 
            'retrievalConfiguration': {
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results, # will fetch top N documents which closely match the query
                    'overrideSearchType' : search_type # usese either SEMANTIC or HYBRID options
                    }
                }
            }
        }
    )
    return response

def print_generation_results(response, print_context = True):
    generated_text = response['output']['text']
    print('Generated FM response:\n')
    pp.pprint(generated_text)
    
    if print_context is True:
        ## print out the source attribution/citations from the original documents to see if the response generated belongs to the context.
        citations = response["citations"]
        contexts = []
        for citation in citations:
            retrievedReferences = citation["retrievedReferences"]
            for reference in retrievedReferences:
                contexts.append(reference["content"]["text"])
    
        print('\n\n\nRetrieved Context:\n')
        pp.pprint(contexts)

def retrieve(query, kb_id, model_arn, max_results, search_type):
    response = bedrock_agent_client.retrieve(
            retrievalQuery={
                'text': query
            },
        knowledgeBaseId= kb_id,
        retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results, # will fetch top N documents which closely match the query
                    'overrideSearchType': search_type
                    }
            }
    )
    return response


def print_results(response):

    print('Retrieved documents:\n')
    
    ## print out the source citations from the original documents
    citations = response['retrievalResults']
    for citation in citations:
        pp.pprint(citation["content"]["text"])


### How hybrid search improves the documents retrieval?

In the following examples, we are going to run the below query with semantic and hybrid search, and observe the documents retrieval and ultimately generated response.

query = `What was the total operating lease cost recognized by the Company during the year ended December 31, 2022?`

The right response for this query as per ground truth QA pair is: 

Expected response = `The Company recognized operating lease costs of $22.4 million during the year ended December 31, 2022.`

**Using semantic search**

In [10]:
query = "What was the total operating lease cost recognized by the Company during the year ended December 31, 2022?"

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 5, search_type='SEMANTIC')

print_generation_results(results)

Generated FM response:

('According to the search results, the Company recognized operating lease '
 'costs of $22.4 million during the year ended December 31, 2022. This amount '
 'was included in occupancy and equipment expenses in the consolidated '
 'statements of operations.')



Retrieved Context:

['Operating Leases   The Company leases office spaces, data centers, and '
 'equipment under various non-cancelable operating leases. These leases have '
 'remaining lease terms ranging from 1 to 10 years, some of which include '
 'options to extend the leases for up to 5 years.   As of December 31, 2022, '
 'the Company had operating lease ROU assets of $87.6 million and operating '
 'lease liabilities of $92.3 million, of which $18.2 million was current and '
 '$74.1 million was non-current. During the year ended December 31, 2022, the '
 'Company recognized operating lease costs of $22.4 million, which were '
 'included in occupancy and equipment expenses in the consolidated stateme

The generated respose using SEMANTIC search for above query is hallucinated.

**Using hyprid search**

In [11]:
# Now let's try with the Hybrid search 

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 5, search_type='HYBRID')

print_generation_results(results)

Generated FM response:

('According to the search results, the Company recognized operating lease '
 'costs of $22.4 million during the year ended December 31, 2022. This amount '
 'was included in occupancy and equipment expenses in the consolidated '
 'statements of operations.')



Retrieved Context:

['Operating Leases   The Company leases office spaces, data centers, and '
 'equipment under various non-cancelable operating leases. These leases have '
 'remaining lease terms ranging from 1 to 10 years, some of which include '
 'options to extend the leases for up to 5 years.   As of December 31, 2022, '
 'the Company had operating lease ROU assets of $87.6 million and operating '
 'lease liabilities of $92.3 million, of which $18.2 million was current and '
 '$74.1 million was non-current. During the year ended December 31, 2022, the '
 'Company recognized operating lease costs of $22.4 million, which were '
 'included in occupancy and equipment expenses in the consolidated stateme

With HYBRID search we got the right response.

### Multi-Query

A different approach to improve the relevance of retrieved documents and subsequently FM generations; is based on multi-query concept. Vector databases find documents similar to your query based distance of its embedding to those docuemtns. In which, it creates problems if the wording used in the query is "closer" to less relevant documents. That are usually resolved by prompt engineering/query tuning.

<img src="images/multi-query.png" width="50%" height="30%">

The approach is based on using an FM to generate multiple queries from different perspectives for a given query. For each query, it retrieves a set of relevant documents then uses a distinct set of document across all queries to get a larger set of potentially relevant documents. By generating multiple queries on the same question, this approach might be able to overcome some of the limitations of the distance-based retrieval and get a more comprehensive set of results.


### How to apply multi queries approach? 

To demonstrate the the multi-queries benefits, we will use the complex question-based approach: 

`What was the total operating lease liabilities and total sublease income of the Company as of December 31, 2022?`

**Using semantic search**

In [12]:
query = "What was the total operating lease liabilities and total sublease income of the Company as of December 31, 2022?"

results = retrieve_and_generate(query = query, kb_id = kb_id, model_arn = model_arn, max_results = 5, search_type='SEMANTIC')

print_generation_results(results)

Generated FM response:

('According to the search results, as of December 31, 2022, the Company had '
 'total operating lease liabilities of $92.3 million, of which $18.2 million '
 'was current and $74.1 million was non-current. The Company also recognized '
 'sublease income of $3.2 million during the year ended December 31, 2022, '
 'which was included in other income in the consolidated statements of '
 'operations.')



Retrieved Context:

['The Company recognizes the lease payments associated with its short-term '
 'leases as an expense on a straight-line basis over the lease term.   '
 'Operating lease ROU assets are presented separately on the consolidated '
 'balance sheets. Finance leases are included in property and equipment on the '
 'consolidated balance sheets. Operating lease liabilities are presented '
 'separately as current and non-current liabilities on the consolidated '
 'balance sheets.   Operating Leases   The Company leases office spaces, data '
 'centers, and 

**Using multi query search**

***Note:*** to simplify the illustration of the multi-query concept, we will use the Langchain library.

In [13]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_aws import BedrockChat
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever
import logging
from langchain_core.prompts.prompt import PromptTemplate


multi_query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is 
    to generate 3 different versions of the given user 
    question to retrieve relevant documents from a vector  database. 
    By generating multiple perspectives on the user question, 
    your goal is to help the user overcome some of the limitations 
    of distance-based similarity search. Provide these alternative 
    questions separated by a single '\n'. Original question: {question}""",
)


# itiating Knowledge bases for Amazon Bedrock as Langchain retriever
retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id= kb_id,
    region_name = region_name,
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": 5
        }
    }
)

# Amazon Bedrock runtime client - to invoke LLM
bedrock_runtime = boto3_session.client("bedrock-runtime",
                              config=bedrock_config)


# prepare the FM inference configurations
inference_modifier = {
    "temperature": 0.0
}

# prepare model id and inference parameters
llm = BedrockChat(
    model_id = model_id,
    client = bedrock_runtime,
    model_kwargs = inference_modifier,
)


# Instantiating the KB as a multi query retreiever
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever = retriever, llm = llm, include_original=True, prompt = multi_query_prompt
)


logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)


unique_docs = retriever_from_llm.get_relevant_documents(query = query)

print(len(unique_docs))

#print(unique_docs)

/Users/mankhanu/Documents/code/github/amazon-bedrock-samples/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use ChatBedrock instead.
  warn_deprecated(
/Users/mankhanu/Documents/code/github/amazon-bedrock-samples/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ["1. What were the Company's total operating lease liabilities and total sublease income recorded on the balance sheet as of December 31, 2022?'", "2. What were the amounts reported for operating lease obligations and sublease rental income in the Company's financial statements for the year ended December 31, 2022?'", "3

20


As you could tell from the count of unique documents, it is richer than the asking the original question only. It is also expected to get an improved answer based on variety of documents retrieve.

Let's contiune with that example and check the final answer:

In [14]:
final_answer_prompt = PromptTemplate(
    input_variables=["documents","question"],
    template="""You are an AI language model assistant. Your task is to answer 
    a given user question based on provided context from a vector database. 
    Use only documents provided in the context to answer the user question. 
    <context>
    {documents}
    </context>
    
    User question: {question}""",
)

query = "What was the total operating lease liabilities and total sublease income of the Company as of December 31, 2022?"
answer= llm.invoke(final_answer_prompt.format(question= query, documents=unique_docs))

pp.pprint(answer.content)

('According to the context provided, as of December 31, 2022:\n'
 '\n'
 'The Company had total operating lease liabilities of $92.3 million, of which '
 '$18.2 million was current and $74.1 million was non-current.\n'
 '\n'
 'The Company recognized sublease income of $3.2 million during the year ended '
 'December 31, 2022, which was included in other income, net, in the '
 'consolidated statements of operations.')


<div class="alert alert-block alert-warning">
<b>Note:</b> Remember to delete KB, OSS index and related IAM roles and policies to avoid incurring any charges.
</div>